# Requirements install

In [1]:
!pip install super-image

     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 22.1 MB 1.5 MB/s 
     |████████████████████████████████| 51.0 MB 153 kB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
     |████████████████████████████████| 76 kB 5.0 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's de

In [2]:
import os
import shutil
from google_drive_downloader import GoogleDriveDownloader as gdd
from google.colab import files
from PIL import Image
from torchvision.transforms import transforms
from pathlib import Path
from super_image.data import EvalDataset, TrainDataset
from super_image import Trainer, TrainingArguments, EdsrModel, EdsrConfig
from super_image import ImageLoader

# Data Preprocess

In [3]:
gdd.download_file_from_google_drive(file_id='1GL_Rh1N-WjrvF_-YOKOyvq0zrV6TF4hb',
                  dest_path='./dataset/HW_data.zip',
                  unzip=True)

Unzipping...Done.


In [4]:
# Train-validation split
TRAIN_IMAGE_LAST_INDEX=262
data_listdir = os.listdir("./dataset/training_hr_images/training_hr_images")

if not os.path.isdir('./dataset/training_hr_images/train'):
    os.mkdir('./dataset/training_hr_images/train')
    os.mkdir('./dataset/training_hr_images/valid')

for i,img_name in enumerate(data_listdir):
  if(i<TRAIN_IMAGE_LAST_INDEX):
    original=os.path.join('./dataset/training_hr_images/training_hr_images', img_name)
    target=os.path.join('./dataset/training_hr_images/train', img_name)
    shutil.copyfile(original, target)
  else:
    original=os.path.join('./dataset/training_hr_images/training_hr_images', img_name)
    target=os.path.join('./dataset/training_hr_images/valid', img_name)
    shutil.copyfile(original, target)

In [5]:
# Create low-resolution images
if not os.path.isdir('./dataset/training_hr_images/train_lr'):
    os.mkdir('./dataset/training_hr_images/train_lr')
    os.mkdir('./dataset/training_hr_images/valid_lr')

# Low-resolution images for Train
train_dicts=[]
train_hr_listdir = os.listdir('./dataset/training_hr_images/train')
for img_name in train_hr_listdir:
  train_dict={'hr':[],'lr':[]}
  img = Image.open('./dataset/training_hr_images/train/'+img_name)
  (w, h) = img.size
  new_img = img.resize((w//3, h//3))
  new_img.save('./dataset/training_hr_images/train_lr/'+img_name)
  train_dict['hr']=['./dataset/training_hr_images/train/'+img_name]
  train_dict['lr']=['./dataset/training_hr_images/train_lr/'+img_name]
  train_dicts.append(train_dict)

# Low-resolution images for validation
valid_dicts=[]
valid_dict={'hr':'','lr':''}
valid_hr_listdir = os.listdir('./dataset/training_hr_images/valid')
for img_name in valid_hr_listdir:
  img = Image.open('./dataset/training_hr_images/valid/'+img_name)
  (w, h) = img.size
  new_img = img.resize((w//3, h//3))
  new_img.save('./dataset/training_hr_images/valid_lr/'+img_name)
  valid_dict['hr']='./dataset/training_hr_images/valid/'+img_name
  valid_dict['lr']='./dataset/training_hr_images/valid_lr/'+img_name
  valid_dicts.append(valid_dict)

In [7]:
# Five crop augmentation
def augment_five_crop(batch, scale=None):
  hr_augment_path = None
  lr_augment_path = None
  if scale is None:
    if len(batch['hr']) > 0:
      scale = get_scale(Image.open(batch['lr'][0]), Image.open(batch['hr'][0]))
  hr_augment_path = Path(batch['hr'][0]).parent / f'augment_x{scale}'
  lr_augment_path = Path(batch['lr'][0]).parent / f'augment_x{scale}'
  hr_augment_path.mkdir(parents=True, exist_ok=True)
  lr_augment_path.mkdir(parents=True, exist_ok=True)
  
  for idx, example in enumerate(batch['hr']):
    hr_path = Path(example)
    lr_path = Path(batch['lr'][idx])
    hr = Image.open(hr_path).convert('RGB')
    outputs_dicts = []
    for aug_idx, hr in enumerate(transforms.FiveCrop(size=(hr.height // 2, hr.width // 2))(hr)):
      hr = hr.resize(((hr.width // scale) * scale, (hr.height // scale) * scale),
                      resample=Image.BICUBIC)
      lr = hr.resize((hr.width // scale, hr.height // scale), resample=Image.BICUBIC)
      hr_file_path = hr_augment_path / f'{hr_path.stem}_{aug_idx}{hr_path.suffix}'
      lr_file_path = lr_augment_path / f'{lr_path.stem}_{aug_idx}{lr_path.suffix}'
      hr.save(hr_file_path, 'PNG')
      lr.save(lr_file_path, 'PNG')
      outputs_dict={'lr':'','hr':''}
      outputs_dict['hr']=hr_file_path.as_posix()
      outputs_dict['lr']=lr_file_path.as_posix()
      outputs_dicts.append(outputs_dict)
  print(outputs_dicts)
  return outputs_dicts

train_dicts_new=[]
for train_dict in train_dicts:
  augment_dict=augment_five_crop(train_dict,scale=3)
  if(len(augment_dict)>1):
    for augment_dict_each in augment_dict:
      train_dicts_new.append(augment_dict_each)

In [10]:
train_dataset = TrainDataset(train_dicts_new,patch_size=12)
eval_dataset = EvalDataset(valid_dicts)

# Model Training

In [11]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=200,
)

config = EdsrConfig( 
    scale=3, 
    n_resblocks=32,  # origimal: 16
    n_feats=256,  # origimal: 64
    res_scale=0.1  # origimal: 1            
)
model = EdsrModel(config)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()

  0%|          | 0/1296 [00:00<?, ?it/s]

scale:3      eval psnr: 23.87     ssim: 0.7092
best epoch: 0, psnr: 23.869629, ssim: 0.709183


  0%|          | 0/1296 [00:00<?, ?it/s]

scale:3      eval psnr: 23.85     ssim: 0.7072


  0%|          | 0/1296 [00:00<?, ?it/s]

...

scale:3      eval psnr: 24.60     ssim: 0.7625


  0%|          | 0/1296 [00:00<?, ?it/s]

scale:3      eval psnr: 24.60     ssim: 0.7627


  0%|          | 0/1296 [00:00<?, ?it/s]

scale:3      eval psnr: 24.60     ssim: 0.7624
best epoch: 199, psnr: 24.603292, ssim: 0.762425


In [12]:
# Download model wegiht and config
!zip -r ./results.zip ./results/
files.download("./results.zip")

  adding: results16/ (stored 0%)
  adding: results16/config.json (deflated 43%)
  adding: results16/pytorch_model_3x.pt (deflated 8%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model prediction

In [13]:
# Generate predicted images
test_listdir = os.listdir("./dataset/testing_lr_images/testing_lr_images")
model_trained = EdsrModel.from_pretrained('/content/results', scale=3)
if not os.path.isdir('./output'):
    os.mkdir('./output')
for i,img_name in enumerate(test_listdir):
  image = Image.open("./dataset/testing_lr_images/testing_lr_images/"+img_name)
  inputs = ImageLoader.load_image(image)
  preds = model_trained(inputs)
  ImageLoader.save_image(preds, './output/'+img_name[:-4]+'_pred.png')

/content/results16/pytorch_model_3x.pt


In [14]:
# Download output images
!zip -r ./output.zip ./output/
files.download("./output.zip")

  adding: output16/ (stored 0%)
  adding: output16/08_pred.png (deflated 0%)
  adding: output16/06_pred.png (deflated 0%)
  adding: output16/05_pred.png (deflated 1%)
  adding: output16/12_pred.png (deflated 0%)
  adding: output16/02_pred.png (deflated 0%)
  adding: output16/10_pred.png (deflated 3%)
  adding: output16/00_pred.png (deflated 1%)
  adding: output16/03_pred.png (deflated 1%)
  adding: output16/04_pred.png (deflated 1%)
  adding: output16/07_pred.png (deflated 4%)
  adding: output16/11_pred.png (deflated 0%)
  adding: output16/01_pred.png (deflated 1%)
  adding: output16/09_pred.png (deflated 2%)
  adding: output16/13_pred.png (deflated 4%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>